In [21]:
import os
import json
import requests
import geojsonio
import time
import geopandas as gpd
import geojson
from os.path import dirname as up
import pandas as pd

import json
import requests
from requests.auth import HTTPBasicAuth

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))



In [22]:
root_path = up(os.getcwd())
meta_path = os.path.join(root_path, 'data/processing_data/planetMeta_2022JanToJun_Kory_edit.csv')
df = pd.read_csv(meta_path)

In [23]:
df = df[df.Useful_Scene=='Y']

In [24]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = ''

# construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

# Setup Planet Data API base URL
URL = "https://api.planet.com/data/v1"

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

# Make a GET request to the Planet Data API
res = session.get(URL)

# Response status code
res.status_code

200

In [25]:
def activating_asset(sid, itemType, productKey):
    
    id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(itemType, sid)
    # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
    result = \
      requests.get(
        id_url,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    # Parse out useful links
    links = result.json()[productKey]["_links"]
    self_link = links["_self"]
    activation_link = links["activate"]

    # Request activation of the asset:
    activate_result = \
      requests.get(
        activation_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )
    
    activation_status_result = \
      requests.get(
        self_link,
        auth=HTTPBasicAuth(API_KEY, '')
      )

    print(activation_status_result.json()["status"])


In [30]:
item_type = "PSScene"
instruKey = "ortho_analytic_8b"

In [31]:
allids = df.id.tolist()

for sID in allids: 
    
    activating_asset(sID, item_type, instruKey)

activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
activating
